# 1.2: Examining the *required* files

이번 워크샵에서 다룰 `kaldi` pipeline은 이 파일에서 설명하는 파일 구조 및 데이터 구조에서 실행됩니다. 

**Note:** 만약 이 pipeline을 **다른 데이터셋**에 적용하고 싶으시다면, 파일 및 데이터 구조를 이 pipeline에서 요구하는 형태로 **반드시** 맞춰야 합니다. 

지금까지 스크립트를 잘 실행하셨다면, 우리가 필요로 하는 모든 파일들은 `raw_data`라는 디렉토리 안에 위치하여야 합니다. `raw_data` 디렉토리에는 다운받은 `*.tar.gz` 파일들과 함께 다음과 같은 파일들을 확인할 수 있습니다. 

 - the `lexicon` file: `librispeech-lexicon.txt`
 - the `transcripts` file: `librispeech-transcripts.txt`
 - the different `language models`: `3-gram.arpa.gz`, `3-gram.pruned.*.arpa.gz`, `4-gram.arpa.gz`

In [ ]:
ls raw_data

다른 데이터셋에 필요한 추가적인 파일들은 `raw_data/LibriSpeech` 디렉토리에 위치해 있습니다. 각각의 데이터셋에 필요한 `audio` 파일 및 `data` 파일이 저장되어 있습니다. 

In [ ]:
ls raw_data/LibriSpeech

## audio files


각각의 데이터셋(*e.g.,* `train`, `dev`, `text` 등)에 해당하는 오디오 파일들은 동일한 위치에 있어야 합니다. (*i.e.,* 다른 하위 폴더 없이 같은 폴더에 위치하여야 합니다.)

In [ ]:
ls raw_data/LibriSpeech/test-clean_audio | head    # look at first 10 files in the directory to exhibit flat structure

오디오 파일의 포맷도 중요합니다. `kaldi`는 `16-bit signed little endian` 포맷으로 오디오 파일이 인코딩 되어있어야 제대로 인식을 합니다. (encoding에 관한 자세한 내용은 [여기](https://wiki.multimedia.cs/index.php/PCM)을 참고하세요.)  

`Sampling rate`는 나중에 직접 데이터를 다룰 때 중요한 역할을 하는 `hyperparameter`입니다. 녹음된 파일은 `16 kHz`인 경우가 많으며, 전화 통화를 녹음했을 경우에는 `8 kHz`인 경우가 많습니다. 이번 워크샵에서는 여러가지 이유로 `librispeech` 데이터를 `8 kHz`로 `downsampling` 하였습니다. 

In [ ]:
file raw_data/LibriSpeech/test-clean_audio/1089-134686-0000.wav

다음 명령어는 각각의 데이터셋에 몇 개의 발화가 포함되어 있는지를 보여줍니다. 

In [ ]:
for part in train-clean-100_audio dev-clean_audio dev-other_audio test-clean_audio test-other_audio; do
    count=$(ls raw_data/LibriSpeech/${part} | wc -l)
    echo "There are ${count} utterances in the ${part} subset"
done

## segments file 
#### (**UNUSED** for `librispeech`)

`librispeech` 데이터는 이미 각각의 발화(~ 2-10초)로 나누어진 음성 파일들로 구성되어 있습니다. `kaldi`는 이렇게 발화가 나누어지지 않은 하나의 긴 데이터도 다룰 수 있습니다. 하지만 그 경우에는 다음과 같은 형식의 `segments` 파일을 필요로 합니다. 

```
[utterance-id] [audio-basename] [utterance-start] [utterance-stop]
[utterance-id] [audio-basename] [utterance-start] [utterance-stop]
[utterance-id] [audio-basename] [utterance-start] [utterance-stop]
```

이와 같은 정보를 통해서 `kaldi`는 해당 파일에서 **각각의 발화**에 해당하는 구간을 따로 나누어서 처리할 수 있게 됩니다. 

## transcript file

모든 음성 파일에 대한 전사(transcription)는 하나의 파일에 다음과 같은 포맷으로 저장되어 있습니다. 

```
[utterance-id] [transcript text]
[utterance-id] [transcript text]
[utterance-id] [transcript text]
```

`utterance-id`는 각각의 발화를 구별하기 위해서 정해진 이름입니다. `librispeech`와 같이 이미 발화가 **구분된** 데이터에서는 음성 파일의 이름이 `utterance-id`와 동일하여도 괜찮습니다. (*i.e.,* 확장자를 제외한 파일 이름)

In [ ]:
head raw_data/librispeech-transcripts.txt

## phones file

이 파일은 `lexicon`에서 사용된 모든 단어의 발음을 구성하는 `phone`을 저장하고 있습니다.

`phone`은 다음과 같은 형식으로 저장되어 있습니다. 

```
[phone_1]
[phone_2]
[phone_3]
```

In [ ]:
head raw_data/librispeech-phones.txt
tail raw_data/librispeech-phones.txt

몇 개의 phone은 뒤에 숫자가 있는 것을 알 수 있습니다. 이 숫자를 이용하여 phone의 강세나 tone을 구별할 수 있습니다. 

`kaldi`는 `silence phones` 또한 필요로 합니다. `silence phones`는 단어가 아닌 발화들을 표시하기 위해서 사용됩니다. 이러한 소리에는 웃음 소리, 기침 소리, 자동차 소음 등이 있습니다. 이번 워크샵에서는 단어가 아닌 것을 발화했을 경우 하나의 `phone`(`SIL`)으로 표현하겠습니다. 각각의 비언어적 소리를 다른 `phone`으로 표현할 수 있습니다. 그러한 경우에는 `transcripts`에도 서로 다르게 전사가 되어있는지 확인하여야 합니다. 

**Note:** 이 `silence phone`은 우리가 pipeline에서 처음으로 다루게 될 `hyperparameter`입니다. 

In [ ]:
cat raw_data/librispeech-phones.txt | grep -A2 -B2 SIL

## lexicon

`lexicon` 파일은 발화에 사용된 모든 단어와 그 `발음`을 담고 있습니다. 

**Note:** `lexicon` 파일에 존재하는 단어들만 `ASR`에서 인식할 수 있습니다. 만약 발화된 단어가 `lexicon`에 존재하지 않는다면, 우리가 구축한 `ASR` 시스템은 해당 단어를 제대로 예측할 수 없습니다. 

`lexicon` 파일의 구조는 다음과 같습니다. 


```
[short_word]    [phone_1] [phone_2] [phone_3]
[longer_word]   [phone_1] [phone_2] [phone_3] [phone_4] [phone_5]
[another_word]  [phone_1] [phone_2] [phone_3]
```


**Note:** 처음 `공백`은 `tab`이며, 나머지는 `space`입니다. <br>
**Note:** `kaldi`는 꼭 알파벳 순으로 정렬된 `lexicon` 파일을 요구하지는 않습니다. 

In [ ]:
head -n5 raw_data/librispeech-lexicon.txt
tail -n5 raw_data/librispeech-lexicon.txt

`kaldi`에서는 한 단어에 하나 이상의 발음이 있을 경우, `lexicon`에서 제공된 발음에 한해서 예측합니다. 

In [ ]:
cat raw_data/librispeech-lexicon.txt | grep "INDIRECTLY"

`lexicon`에는 또한 사전에 존재하지 않는 단어를 처리하기 위한 `<unk>`가 있습니다. `kaldi`는 제대로 인식/예측할 수 없는 모든 단어를 처리하기 위한 `placeholder`를 요구하므로, `<unk>`가 추가되었습니다. `<unk>`의 발음으로는 `nonsilence phone`인 `SIL`이 설정되어 있습니다. 

In [ ]:
cat raw_data/librispeech-lexicon.txt | grep "<unk>"

## language model


`language model`은 `ARPA` 포맷으로 작성되어 있습니다. (자세한 내용은 다음 노트북에서 다룹니다.)

In [ ]:
head raw_data/3-gram.pruned.3e-7.arpa
echo "..."
grep -A4 -E '\\2-grams' raw_data/3-gram.pruned.3e-7.arpa
echo "..."
grep -A4 -E '\\3-grams' raw_data/3-gram.pruned.3e-7.arpa
echo "..."
tail raw_data/3-gram.pruned.3e-7.arpa

**Note:** `language model`은 압축된 상태로도 사용이 가능하지만, 나중에 `piping` `gzip -d`와 같은 명령어를 이용해 다시 압축을 풀어주어야 합니다. 편리한 사용을 위해 이전 노트북에서 `language model`은 모두 압축을 풀어두었습니다.

`librispeech` 데이터에 있는 각각의 데이터셋에 필요한 `language model`을 확인할 수 있습니다. 

In [ ]:
ls raw_data | grep "arpa"